<a href="https://colab.research.google.com/github/Janebhop/fraud-detection-course/blob/main/notebook/chapter3/Part_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Part Model

In [8]:
### mount drive
import os,sys
from google.colab import drive
drive.mount('/content/drive')
nb_path = '/content/notebooks'
# %cd /content/
### check current dir 
!pwd
### simulate drive from Colab Notebooks to
os.symlink('/content/drive/My Drive/fraud', nb_path)
sys.path.insert(0, nb_path) 

Mounted at /content/drive
/content


In [ ]:
a = []
while(1):
    a.append('1')

In [2]:
import pandas as pd
import numpy as np

### Import Data

In [3]:
df_test = pd.read_csv('/content/drive/My Drive/fraud/PS_20174392719_1491204439457_log.csv', nrows=10)


float_cols = [c for c in df_test if df_test[c].dtype == "float64"]
float32_cols = {c: np.float32 for c in float_cols}

In [4]:
data = pd.read_csv('/content/drive/My Drive/fraud/PS_20174392719_1491204439457_log.csv', engine='c', dtype=float32_cols)
data

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,1,PAYMENT,9.839640e+03,C1231006815,170136.000,160296.359375,M1979787155,0.000000e+00,0.000,0,0
1,1,PAYMENT,1.864280e+03,C1666544295,21249.000,19384.720703,M2044282225,0.000000e+00,0.000,0,0
2,1,TRANSFER,1.810000e+02,C1305486145,181.000,0.000000,C553264065,0.000000e+00,0.000,1,0
3,1,CASH_OUT,1.810000e+02,C840083671,181.000,0.000000,C38997010,2.118200e+04,0.000,1,0
4,1,PAYMENT,1.166814e+04,C2048537720,41554.000,29885.859375,M1230701703,0.000000e+00,0.000,0,0
...,...,...,...,...,...,...,...,...,...,...,...
6362615,743,CASH_OUT,3.396821e+05,C786484425,339682.125,0.000000,C776919290,0.000000e+00,339682.125,1,0
6362616,743,TRANSFER,6.311410e+06,C1529008245,6311409.500,0.000000,C1881841831,0.000000e+00,0.000,1,0
6362617,743,CASH_OUT,6.311410e+06,C1162922333,6311409.500,0.000000,C1365125890,6.848884e+04,6379898.000,1,0
6362618,743,TRANSFER,8.500025e+05,C1685995037,850002.500,0.000000,C2080388513,0.000000e+00,0.000,1,0


### Preprocessing

In [5]:
# cleaning data 
# checking null values

data.isnull().values.any()

False

In [6]:
data.head()

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,1,PAYMENT,9839.639648,C1231006815,170136.0,160296.359375,M1979787155,0.0,0.0,0,0
1,1,PAYMENT,1864.280029,C1666544295,21249.0,19384.720703,M2044282225,0.0,0.0,0,0
2,1,TRANSFER,181.000000,C1305486145,181.0,0.000000,C553264065,0.0,0.0,1,0
3,1,CASH_OUT,181.000000,C840083671,181.0,0.000000,C38997010,21182.0,0.0,1,0
4,1,PAYMENT,11668.139648,C2048537720,41554.0,29885.859375,M1230701703,0.0,0.0,0,0


In [7]:
# feature selection

data = data.drop(['nameOrig','nameDest'],axis = 1)
data

,step,type,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,1,PAYMENT,9.839640e+03,170136.000,160296.359375,0.000000e+00,0.000,0,0
1,1,PAYMENT,1.864280e+03,21249.000,19384.720703,0.000000e+00,0.000,0,0
2,1,TRANSFER,1.810000e+02,181.000,0.000000,0.000000e+00,0.000,1,0
3,1,CASH_OUT,1.810000e+02,181.000,0.000000,2.118200e+04,0.000,1,0
4,1,PAYMENT,1.166814e+04,41554.000,29885.859375,0.000000e+00,0.000,0,0
...,...,...,...,...,...,...,...,...,...
6362615,743,CASH_OUT,3.396821e+05,339682.125,0.000000,0.000000e+00,339682.125,1,0
6362616,743,TRANSFER,6.311410e+06,6311409.500,0.000000,0.000000e+00,0.000,1,0
6362617,743,CASH_OUT,6.311410e+06,6311409.500,0.000000,6.848884e+04,6379898.000,1,0
6362618,743,TRANSFER,8.500025e+05,850002.500,0.000000,0.000000e+00,0.000,1,0


In [9]:
pd.get_dummies(data['type'], prefix='Type')

,Type_CASH_IN,Type_CASH_OUT,Type_DEBIT,Type_PAYMENT,Type_TRANSFER
0,0,0,0,1,0
1,0,0,0,1,0
2,0,0,0,0,1
3,0,1,0,0,0
4,0,0,0,1,0
...,...,...,...,...,...
6362615,0,1,0,0,0
6362616,0,0,0,0,1
6362617,0,1,0,0,0
6362618,0,0,0,0,1


In [ ]:
# feature engineering

from sklearn.preprocessing import OneHotEncoder 
from sklearn.compose import ColumnTransformer 
   
# creating one hot encoder object with categorical feature 1 
# indicating the second column 
columnTransformer = ColumnTransformer([('encoder', 
                                        OneHotEncoder(), 
                                        [1])], 
                                      remainder='passthrough') 
  
data = pd.DataFrame(np.array(columnTransformer.fit_transform(data), dtype = np.str))
data

In [10]:
columnTransformer

ColumnTransformer(n_jobs=None, remainder='passthrough', sparse_threshold=0.3,
                  transformer_weights=None,
                  transformers=[('encoder',
                                 OneHotEncoder(categories='auto', drop=None,
                                               dtype=<class 'numpy.float64'>,
                                               handle_unknown='error',
                                               sparse=True),
                                 [1])],
                  verbose=False)

In [ ]:
# change column name
data_enc.rename(columns={0: "CASH_IN", 1: "CASH_OUT",2: "DEBIT",3: "PAYMENT",4: "TRANSFER",5: "step",6: "amount",7: "oldbalanceOrig",8: "newbalanceOrig",9: "oldbalanceDest",10: "newbalanceDest",11: "isFraud",12: "isFlaggedFraud",},inplace = True)

In [ ]:
# splitting data

# fraud and no fraud transactions are separated by 'isFraud' column
dataF = data_enc[data_enc['isFraud'] == '1.0']
dataNF = data_enc[data_enc['isFraud'] == '0.0']

# split fraud transaction into 80 percent and 20 percent sessions
mskF = np.random.rand(len(dataF)) < 0.8
trainF = dataF[mskF]
testF = dataF[~mskF]

# split no fraud transaction into 80 percent and 20 percent sessions
mskNF = np.random.rand(len(dataNF)) < 0.8
trainNF = dataNF[mskNF]
testNF = dataNF[~mskNF]

# 
train_data = pd.concat([trainF,trainNF]).reset_index(drop = True)
test_data = pd.concat([testF,testNF]).reset_index(drop = True)

#create input data attributes by dropped irrevalent and target column 
train_feature = train_data.drop(['isFraud','isFlaggedFraud'],axis = 1)
test_feature = test_data.drop(['isFraud','isFlaggedFraud'],axis = 1)

train_label = train_data['isFraud']
test_label = test_data['isFraud']

In [ ]:
# feature scaling

from sklearn import preprocessing
scaler = preprocessing.StandardScaler()
train_feature_scaled =scaler.fit_transform(train_feature)
train_feature_scaled[1]

test_feature_scaled = scaler.transform(test_feature)

## Logistic Regression

Logistic regression is used to describe data and to explain the relationship between one dependent binary variable and one or more nominal, ordinal, interval or ratio-level independent variables
    
    class sklearn.linear_model.LogisticRegression (penalty='l2', *, dual=False, tol=0.0001, C=1.0, fit_intercept=True, intercept_scaling=1, class_weight=None, random_state=None, solver='lbfgs', max_iter=100, multi_class='auto', verbose=0, warm_start=False, n_jobs=None, l1_ratio=None)
    
https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html

In [ ]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(random_state=0)
clf.fit(train_feature_scaled,train_label)

# predict train data
lr_predY_train = clf.predict(train_feature_scaled)

In [ ]:
# predict test data
lr_predY = clf.predict(test_feature_scaled)
lr_predY[0:10]

## XGBoost 

XGBoost stands for Extreme Gradient Boosting; it is a specific implementation of the Gradient Boosting method which uses more accurate approximations to find the best tree model. It employs a number of nifty tricks that make it exceptionally successful, particularly with structured data.

XGBoost's hyperparameters

    -learning_rate: step size shrinkage used to prevent overfitting. Range is [0,1]
    -max_depth: determines how deeply each tree is allowed to grow during any boosting round.
    -subsample: percentage of samples used per tree. Low value can lead to underfitting.
    -colsample_bytree: percentage of features used per tree. High value can lead to overfitting.
    -n_estimators: number of trees you want to build.
    -objective: determines the loss function to be used like reg:linear for regression problems, reg:logistic for classification problems with only decision, binary:logistic for classification problems with probability.

XGBoost also supports regularization parameters to penalize models as they become more complex and reduce them to simple (parsimonious) models.

    -gamma: controls whether a given node will split based on the expected reduction in loss after the split. A higher value leads to fewer splits. Supported only for tree-based learners.
    -alpha: L1 regularization on leaf weights. A large value leads to more regularization.
    -lambda: L2 regularization on leaf weights and is smoother than L1 regularization.

In [ ]:
from xgboost.sklearn import XGBClassifier
xgbc = XGBClassifier()

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=1, gamma=0, learning_rate=0.1,
       max_delta_step=0, max_depth=3, min_child_weight=1, missing=None,
       n_estimators=100, n_jobs=1, nthread=None,
       objective='multi:softprob', random_state=0, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, seed=None, silent=None,
       subsample=1, verbosity=1) 

xgbc.fit(train_feature_scaled, train_label)

# predict train data
xgb_predY_train = xgbc.predict(train_feature_scaled)

[00:01:56] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\gbm\gbtree.cc:139: Tree method is automatically selected to be 'approx' for faster speed. To use old behavior (exact greedy algorithm on single machine), set tree_method to 'exact'.


In [ ]:
# predict test data
xgb_predY = xgbc.predict(test_feature_scaled)
xgb_predY[0:10]

array(['1.0', '1.0', '0.0', '1.0', '1.0', '1.0', '1.0', '1.0', '0.0',
       '1.0'], dtype=object)

## Model Evaluation

### Confusion Matrix

confusion matrix, also known as an error matrix, is a specific table layout that allows visualization of the performance of an algorithm, typically a supervised learning one (in unsupervised learning it is usually called a matching matrix). Each row of the matrix represents the instances in a predicted class while each column represents the instances in an actual class (or vice versa)

![image.png](attachment:image.png)


https://en.wikipedia.org/wiki/Confusion_matrix

--------------------------------------------
Accuracy: Overall, how often is the classifier correct?
    
    Accuracy = (TP+TN)/total = (TP+TN)/(TP+TN+FN+FP)

Precision: When it predicts yes, how often is it correct?

    Precision = TP/predicted yes = TP/(TP+FP)
    
Recall: When it's actually yes, how often does it predict yes? (True Positive Rate)

    Recall = TP/actual yes = TP/(TP+FN)
    
F1 Score: This is a weighted average of the recall and precision

    F1 Score = (2*(Precision*Recall))/(Precision+Recall)

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score, recall_score, precision_score, f1_score

accuracy_scorelr_train = accuracy_score(train_label,lr_predY_train)
precision_scorelr_train = precision_score(train_label,lr_predY_train,average="binary",pos_label="1.0")
recall_scorelr_train = recall_score(train_label,lr_predY_train,average="binary",pos_label="1.0")
f1_scorelr_train = f1_score(train_label, lr_predY_train, average='macro')

print("Accuracy_score_train : ",accuracy_scorelr_train)
print("Precision_score_train : ",precision_scorelr_train)
print("Recall_score_train : ",recall_scorelr_train)
print("F1_score_train : ",f1_scorelr_train)

Accuracy_score_train :  0.999218865604206
Precision_score_train :  0.9233791748526523
Recall_score_train :  0.42981252857796065
F1_score_train :  0.7930962634054941


In [ ]:
accuracy_scorelr = accuracy_score(test_label,lr_predY)
precision_scorelr = precision_score(test_label,lr_predY,average="binary",pos_label="1.0")
recall_scorelr = recall_score(test_label,lr_predY,average="binary",pos_label="1.0")
f1_scorelr = f1_score(test_label, lr_predY, average='macro')

print("Accuracy_score : ",accuracy_scorelr)
print("Precision_score : ",precision_scorelr)
print("Recall_score : ",recall_scorelr)
print("F1_score : ",f1_scorelr)

Accuracy_score_train :  0.9992487441781599
Precision_score_train :  0.9284833538840938
Recall_score_train :  0.45581113801452783
F1_score_train :  0.8055367302452809


In [ ]:
cm_lr = confusion_matrix(lr_predY,test_label,labels=["1.0", "0.0"])
cm_lr

array([[    753,      58],
       [    899, 1272157]], dtype=int64)

In [ ]:
accuracy_scorexgb_train = accuracy_score(train_label,xgb_predY_train)
precision_scorexgb_train = precision_score(train_label,xgb_predY_train,average="binary",pos_label="1.0")
recall_scorexgb_train = recall_score(train_label,xgb_predY_train,average="binary",pos_label="1.0")
f1_scorexgb_train = f1_score(train_label, xgb_predY_train, average='macro')

print("Accuracy_score_train : ",accuracy_scorexgb_train)
print("Precision_score_train : ",precision_scorexgb_train)
print("Recall_score_train : ",recall_scorexgb_train)
print("F1_score_train : ",f1_scorexgb_train)

Accuracy_score_train :  0.999848882427581
Precision_score_train :  0.9829886591060707
Recall_score_train :  0.898338667885993
F1_score_train :  0.9693418028293759


In [ ]:
accuracy_scorexgb = accuracy_score(test_label,xgb_predY)
precision_scorexgb = precision_score(test_label,xgb_predY,average="binary",pos_label="1.0")
recall_scorexgb = recall_score(test_label,xgb_predY,average="binary",pos_label="1.0")
f1_scorelxgb = f1_score(test_label, xgb_predY, average='macro')

print("Accuracy_score_xgb : ",accuracy_scorexgb)
print("Precision_score_xgb : ",precision_scorexgb)
print("Recall_score_xgb : ",recall_scorexgb)
print("F1_score_xgb : ",f1_scorelxgb)

Accuracy_score_xgb :  0.999784906901584
Precision_score_xgb :  0.9674355495251018
Recall_score_xgb :  0.8631961259079903
F1_score_xgb :  0.9561201849782245


In [ ]:
cm_xgb = confusion_matrix(xgb_predY,test_label,labels=["1.0", "0.0"])
cm_xgb

array([[   1426,      48],
       [    226, 1272167]], dtype=int64)

In [ ]:
1426/(1426+226)

0.8631961259079903

## Random OverSampling 

Random oversampling involves randomly duplicating examples from the minority class and adding them to the training dataset.

This technique can be effective for those machine learning algorithms that are affected by a skewed distribution and where multiple duplicate examples for a given class can influence the fit of the model

In [ ]:
from imblearn.over_sampling import RandomOverSampler 
ros = RandomOverSampler()
train_feature_resampled, train_label_resampled = ros.fit_sample(train_feature, train_label)

In [ ]:
scaler = preprocessing.StandardScaler()
train_feature_scaled =scaler.fit_transform(train_feature_resampled)
test_feature_scaled = scaler.transform(test_feature)

In [ ]:
clf_ros = LogisticRegression(random_state=0)
clf_ros.fit(train_feature_scaled,train_label_resampled)

C:\Users\saranya.hir\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(random_state=0)

In [ ]:
lr_predY_ros_train = clf_ros.predict(train_feature_scaled)

In [ ]:
accuracy_scorelr_ros_train = accuracy_score(train_label_resampled,lr_predY_ros_train)
f1_scorelr_ros_train = f1_score(train_label_resampled, lr_predY_ros_train, average='macro')
precision_scorelr_ros_train = precision_score(train_label_resampled,lr_predY_ros_train,average="binary",pos_label="1.0")
recall_scorelr_ros_train = recall_score(train_label_resampled,lr_predY_ros_train,average="binary",pos_label="1.0")

print("accuracy score_train = ",accuracy_scorelr_ros_train)
print("precision score_train = ",precision_scorelr_ros_train)
print("recall score_train = ",recall_scorelr_ros_train)
print("f1 score score_train = ",f1_scorelr_ros_train)

accuracy score_train =  0.9489807744374869
precision score_train =  0.9483789882242392
recall score_train =  0.9496518431416995
f1 score score_train =  0.9489807514618251


In [ ]:
lr_predY_ros = clf_ros.predict(test_feature_scaled)

In [ ]:
accuracy_scorelr_ros = accuracy_score(test_label,lr_predY_ros)
f1_scorelr_ros = f1_score(test_label, lr_predY_ros, average='macro')
precision_scorelr_ros = precision_score(test_label,lr_predY_ros,average="binary",pos_label="1.0")
recall_scorelr_ros = recall_score(test_label,lr_predY_ros,average="binary",pos_label="1.0")

print("accuracy score_ros = ",accuracy_scorelr_ros)
print("precision score_ros = ",precision_scorelr_ros)
print("recall score_ros = ",recall_scorelr_ros)
print("f1 score score_ros = ",f1_scorelr_ros)

accuracy score_ros =  0.9479898607939448
precision score_ros =  0.023260619299229564
recall score_ros =  0.9539951573849879
f1 score score_ros =  0.5093402975638798


In [ ]:
cm_lr_ros = confusion_matrix(lr_predY_ros,test_label,labels=["1.0", "0.0"])
cm_lr_ros

array([[   1576,   66178],
       [     76, 1206037]], dtype=int64)

In [ ]:
1577/(1577+75)

0.9546004842615012

## Random Undersampling

Random undersampling involves randomly selecting examples from the majority class to delete from the training dataset.

This has the effect of reducing the number of examples in the majority class in the transformed version of the training dataset. This process can be repeated until the desired class distribution is achieved, such as an equal number of examples for each class

A limitation of undersampling is that examples from the majority class are deleted that may be useful, important, or perhaps critical to fitting a robust decision boundary. Given that examples are deleted randomly, there is no way to detect or preserve “good” or more information-rich examples from the majority class.

https://machinelearningmastery.com/random-oversampling-and-undersampling-for-imbalanced-classification/

In [ ]:
from imblearn.under_sampling import RandomUnderSampler
rus = RandomUnderSampler()
train_feature_resampled, train_label_resampled = rus.fit_sample(train_feature,train_label)

In [ ]:
scaler = preprocessing.StandardScaler()
train_feature_scaled =scaler.fit_transform(train_feature_resampled)
test_feature_scaled = scaler.transform(test_feature)

In [ ]:
clf_rus = LogisticRegression(random_state=0)
clf_rus.fit(train_feature_scaled,train_label_resampled)

LogisticRegression(random_state=0)

In [ ]:
lr_predY_rus_train = clf_rus.predict(train_feature_scaled)

In [ ]:
accuracy_scorelr_rus_train = accuracy_score(train_label_resampled,lr_predY_rus_train)
f1_scorelr_rus_train = f1_score(train_label_resampled, lr_predY_rus_train, average='macro')
precision_scorelr_rus_train = precision_score(train_label_resampled,lr_predY_rus_train,average="binary",pos_label="1.0")
recall_scorelr_rus_train = recall_score(train_label_resampled,lr_predY_rus_train,average="binary",pos_label="1.0")

print("accuracy score_rus_train = ",accuracy_scorelr_rus_train)
print("precision score_rus_train = ",precision_scorelr_rus_train)
print("recall score_rus_train = ",recall_scorelr_rus_train)
print("f1 score score_rus_train = ",f1_scorelr_rus_train)

accuracy score_rus_train =  0.9263831732967536
precision score_rus_train =  0.9296575026877592
recall score_rus_train =  0.9225727785398568
f1 score score_rus_train =  0.926382104430562


In [ ]:
lr_predY_rus = clf_rus.predict(test_feature_scaled)

In [ ]:
accuracy_scorelr_rus = accuracy_score(test_label,lr_predY_rus)
f1_scorelr_rus = f1_score(test_label, lr_predY_rus, average='macro')
precision_scorelr_rus = precision_score(test_label,lr_predY_rus,average="binary",pos_label="1.0")
recall_scorelr_rus = recall_score(test_label,lr_predY_rus,average="binary",pos_label="1.0")

print("accuracy score_rus = ",accuracy_scorelr_rus)
print("precision score_rus = ",precision_scorelr_rus)
print("recall score_rus = ",recall_scorelr_rus)
print("f1 score_rus = ",f1_scorelr_rus)

accuracy score_rus =  0.9297728883784571
precision score_rus =  0.01658225060559348
recall score_rus =  0.9116222760290557
f1 score_rus =  0.4980679958781676


In [ ]:
cm_lr_rus = confusion_matrix(lr_predY_rus,test_label,labels=["1.0", "0.0"])
cm_lr_rus

array([[   1506,   89314],
       [    146, 1182901]], dtype=int64)

In [ ]:
1505/(1505+147)

0.9110169491525424